In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_performance_radar_chart():
    """
    该函数用于绘制三个模型在交叉验证中的平均性能雷达图。
    每个子图包含训练集和验证集的性能对比，以直观展示模型的综合表现和过拟合情况。
    """
    
    # 0. 定义输出路径
    # 使用r-string来避免路径中的反斜杠问题
    output_folder = r"C:\Users\Michael Wang\OneDrive\小论文\心血管疾病预测\建模_数据预处理\output"
    # 确保输出文件夹存在，如果不存在则创建
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, "performance_radar_chart.png")

    # 1. 定义数据
    # 创建一个包含所有模型性能数据的DataFrame，方便管理
    data = {
        'Metric': ['Accuracy', 'F1 Score', 'AUC'],
        # 基线模型 (Baseline Model)
        'Baseline_Train': [0.975530, 0.982487, 0.999281],
        'Baseline_Val':   [0.952686, 0.966214, 0.991191],
        # SMOTE 增强模型
        'SMOTE_Train':    [0.991725, 0.991815, 0.999818],
        'SMOTE_Val':      [0.968999, 0.977387, 0.992894],
        # WGAN-GP 增强模型
        'WGAN_GP_Train':  [0.998165, 0.998713, 0.999980],
        'WGAN_GP_Val':    [0.959243, 0.970592, 0.990069]
    }
    df = pd.DataFrame(data)

    # 2. 准备雷达图的标签和角度
    labels = df['Metric'].values
    num_vars = len(labels)
    
    # 计算每个标签的角度
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    # 为了让雷达图闭合，需要将第一个角度值追加到末尾
    angles += angles[:1]

    # 3. 创建绘图区域
    # 创建一个1行3列的子图布局，并指定所有子图都使用极坐标系
    fig, axes = plt.subplots(figsize=(20, 6), ncols=3, subplot_kw=dict(projection='polar'))
    # fig.suptitle('Comparative Analysis of Model Performance in Cross-Validation', fontsize=18, y=1.05)

    # 定义模型信息列表，方便循环绘图
    models_info = [
        {'title': 'Baseline Model', 'train_col': 'Baseline_Train', 'val_col': 'Baseline_Val'},
        {'title': 'SMOTE Augmented Model', 'train_col': 'SMOTE_Train', 'val_col': 'SMOTE_Val'},
        {'title': 'WGAN-GP Augmented Model', 'train_col': 'WGAN_GP_Train', 'val_col': 'WGAN_GP_Val'}
    ]

    # 4. 循环绘制每个模型的雷达图
    for i, model in enumerate(models_info):
        ax = axes[i]
        
        # 提取当前模型的数据
        train_values = df[model['train_col']].tolist()
        train_values += train_values[:1] # 闭合雷达图
        
        val_values = df[model['val_col']].tolist()
        val_values += val_values[:1] # 闭合雷达图

        # 设置Y轴的范围，可以根据数据微调，这里设置为0.95到1.0
        ax.set_ylim(0.95, 1.0)
        
        # 绘制角度线和标签
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, size=15)
        ax.tick_params(axis='y', labelsize=15)

        # 绘制训练集性能（蓝色虚线轮廓）
        ax.plot(angles, train_values, color='blue', linewidth=2, linestyle='dashed', label='CV Train Avg.')
        
        # 绘制验证集性能（红色实线轮廓并填充）
        ax.plot(angles, val_values, color='red', linewidth=2, linestyle='solid', label='CV Validation Avg.')
        ax.fill(angles, val_values, 'red', alpha=0.25)
        
        # 设置子图标题
        ax.set_title(model['title'], size=15, pad=20)
        
    # 5. 添加图例并调整布局
    # 在最后一个子图下方添加一个统一的图例
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, fontsize=15, bbox_to_anchor=(0.5, -0.05))
    
    plt.tight_layout(pad=3.0)
    
    # 保存图像到指定路径
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"性能雷达图已保存至: {output_path}")

    plt.show()

# 执行绘图函数
if __name__ == '__main__':
    plot_performance_radar_chart()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

def plot_confusion_matrices():
    """
    该函数用于并排绘制三个模型在最终测试集上的混淆矩阵。
    使用Seaborn的热力图进行可视化，以清晰展示模型的最终分类性能。
    """
    
    # 0. 定义输出路径
    # 使用r-string来避免路径中的反斜杠问题
    output_folder = r"C:\Users\Michael Wang\OneDrive\小论文\心血管疾病预测\建模_数据预处理\output"
    # 确保输出文件夹存在，如果不存在则创建
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, "confusion_matrices_comparison.png")

    # 1. 定义混淆矩阵数据
    # 矩阵格式: [[TN, FP], [FN, TP]]
    # TN: True Negative, FP: False Positive, FN: False Negative, TP: True Positive
    
    # 基线模型 (Baseline Model)
    cm_baseline = np.array([[45, 3], [0, 106]])
    
    # SMOTE 增强模型
    cm_smote = np.array([[48, 0], [0, 106]])
    
    # WGAN-GP 增强模型
    cm_wgangp = np.array([[46, 2], [0, 106]])
    
    # 将所有模型信息存入列表，方便循环处理
    models_info = [
        {'title': 'Baseline Model', 'matrix': cm_baseline},
        {'title': 'SMOTE Augmented Model', 'matrix': cm_smote},
        {'title': 'WGAN-GP Augmented Model', 'matrix': cm_wgangp}
    ]
    
    # 定义类别标签
    class_labels = ['Absence', 'Presence']

    # 2. 创建绘图区域
    # 创建一个1行3列的子图布局，并设置合适的尺寸
    fig, axes = plt.subplots(1, 3, figsize=(21, 6))
    # fig.suptitle('Confusion Matrices on the Final Hold-Out Test Set', fontsize=18, y=1.02)

    # 3. 循环绘制每个模型的混淆矩阵
    for i, model in enumerate(models_info):
        ax = axes[i]
        
        # 使用Seaborn的heatmap函数绘制热力图
        sns.heatmap(model['matrix'], 
                    ax=ax, 
                    annot=True,      # 在格子上显示数字
                    fmt='d',         # 数字格式为整数
                    cmap='Blues',    # 颜色映射方案
                    linewidths=.5,   # 格子间的线条宽度
                    annot_kws={"size": 20}) # 注释文字的大小
        
        # 设置子图的标题和标签
        ax.set_title(model['title'], fontsize=16)
        ax.set_xlabel('Predicted Label', fontsize=16, labelpad=20)
        ax.set_ylabel('True Label', fontsize=16)
        
        # 设置坐标轴的刻度标签
        ax.set_xticklabels(class_labels, rotation=0, ha='center',fontsize=13)
        ax.set_yticklabels(class_labels, rotation=0,fontsize=13)

    # 4. 调整整体布局并保存/显示图像
    plt.tight_layout(pad=3.0)
    
    # 保存图像到指定路径
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"混淆矩阵图已保存至: {output_path}")
    
    plt.show()

# 执行绘图函数
if __name__ == '__main__':
    plot_confusion_matrices()
